In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import os
import time
from collections import deque
from numba import njit
from joblib import Parallel, delayed

# =========================================================
# GRAPH BUILDERS
# =========================================================
def make_graph(topo, N, z, seed):
    if topo == "BA":
        m = int(z / 2)
        return nx.barabasi_albert_graph(N, m, seed=seed)
    else:
        return nx.erdos_renyi_graph(N, z/(N-1), seed=seed)

def graph_to_csr(G):
    G = nx.convert_node_labels_to_integers(G)
    N = G.number_of_nodes()
    adj_f = []
    adj_o = [0]
    for i in range(N):
        neigh = list(G.neighbors(i))
        adj_f.extend(neigh)
        adj_o.append(len(adj_f))
    return np.array(adj_f, dtype=np.int32), np.array(adj_o, dtype=np.int32), (np.array(adj_o[1:]) - np.array(adj_o[:-1])).astype(np.int32)

def build_influence_layers_bfs(G_i, N, Lmax):
    adj = [list(G_i.neighbors(i)) for i in range(N)]
    inf_indices = np.zeros((Lmax, N, N), dtype=np.int32)
    inf_counts = np.zeros((Lmax, N), dtype=np.int32)
    for src in range(N):
        dist = np.full(N, -1, dtype=np.int32)
        dist[src] = 0
        q = deque([src])
        while q:
            u = q.popleft()
            if dist[u] == Lmax: continue
            for v in adj[u]:
                if dist[v] == -1:
                    dist[v] = dist[u] + 1
                    q.append(v)
        for j in range(N):
            d = dist[j]
            if 0 < d <= Lmax:
                l, c = d - 1, inf_counts[d-1, src]
                inf_indices[l, src, c] = j
                inf_counts[l, src] += 1
    return inf_indices, inf_counts

# =========================================================
# CORE SIMULATION (AVEC CODES D'ARRÊT)
# =========================================================
@njit(cache=True, fastmath=True)
def run_simulation_exhaustive_v2(
    adj_f, adj_o, deg_f, inf_indices, inf_counts,
    alpha_weights, theta, b, steps, update_rule, K, seed, sampling_rate=100):
    
    np.random.seed(seed)
    N = deg_f.shape[0]
    L = alpha_weights.shape[0]
    
    strat = np.random.randint(0, 2, N).astype(np.float64)
    vig = np.zeros(N, dtype=np.float64)
    for i in range(N):
        if strat[i] == 1.0 and np.random.random() < 0.5: vig[i] = 1.0

    # Critères de stabilité
    history_window = 300
    burn_in = 1200
    tol_std = 8e-4
    history_rho = np.zeros(history_window)
    filled = 0

    rho_history = np.zeros(steps // sampling_rate)
    
    influences = np.empty(N); T_eff = np.empty(N); payoffs = np.empty(N); new_strat = np.empty(N)

    for t in range(steps):
        if t % sampling_rate == 0:
            rho_history[t // sampling_rate] = strat.mean()

        # 1. Influence
        for i in range(N):
            s = 0.0
            for l in range(L):
                cnt = inf_counts[l, i]
                if cnt > 0:
                    v_sum = 0.0
                    for k in range(cnt): v_sum += vig[inf_indices[l, i, k]]
                    s += alpha_weights[l] * (v_sum / cnt)
            influences[i] = s

        # 2. Vigilance
        for i in range(N):
            vig[i] = 1.0 if (strat[i] == 1.0 and influences[i] >= theta) else 0.0

        # 3. Payoffs
        for i in range(N):
            T_eff[i] = 1.0 + (b - 1.0) * (1.0 - influences[i])
            payoffs[i] = 0.0
        for i in range(N):
            for idx in range(adj_o[i], adj_o[i + 1]):
                j = adj_f[idx]
                if strat[j] == 1.0:
                    payoffs[i] += 1.0 if strat[i] == 1.0 else T_eff[i]

        # 4. Actualisation
        for i in range(N):
            new_strat[i] = strat[i]
            ki = deg_f[i]
            if ki == 0: continue
            j = adj_f[adj_o[i] + np.random.randint(0, ki)]
            
            if update_rule == 0: # Proportional
                if payoffs[j] > payoffs[i]:
                    phi = max(ki, deg_f[j]) * max(T_eff[i], T_eff[j])
                    if phi > 0 and np.random.random() < (payoffs[j] - payoffs[i]) / phi:
                        new_strat[i] = strat[j]
            else: # Fermi
                p = 1.0 / (1.0 + np.exp((payoffs[i] - payoffs[j]) / max(K, 1e-12)))
                if np.random.random() < p:
                    new_strat[i] = strat[j]
        
        strat[:] = new_strat[:]
        rho = strat.mean()

        # --- CONDITIONS D'ARRÊT ---
        # A) FIXATION
        if rho == 0.0 or rho == 1.0:
            rho_history[t // sampling_rate:] = rho
            return rho, t, 0, rho_history # Code 0: Fixation

        # Enregistrement pour stabilité
        history_rho[t % history_window] = rho
        filled = min(filled + 1, history_window)

        # B) ÉQUILIBRE (STATIONNAIRE)
        if t > burn_in and filled == history_window and (t % 50 == 0):
            std = np.std(history_rho)
            if std < tol_std:
                rho_history[t // sampling_rate:] = history_rho.mean()
                return history_rho.mean(), t, 1, rho_history # Code 1: Équilibre

    # C) TEMPS EXPIRÉ
    return strat.mean(), steps, 2, rho_history # Code 2: Max steps

# =========================================================
# CONFIGURATION ET BOUCLE SUR TOPOLOGIES
# =========================================================
N, REPS, steps, sampling_rate = 500, 100, 30000, 100
theta, L = 0.4, 1
B_RANGE = np.around(np.arange(1, 2.1, 0.1), 2)
CASOS = [("Proportional", 0, 0.0), ("Fermi K=0.1", 1, 0.1)]
TOPOLOGIES = [("BA", 4), ("ER", 16)]

for topo_name, z_val in TOPOLOGIES:
    print(f"\n--- Topología: {topo_name} Z={z_val} ---")
    
    # Préchauffage graphe
    Gg = make_graph(topo_name, N, z_val, seed=42)
    af, ao, deg = graph_to_csr(Gg)
    inf_idx, inf_cnt = build_influence_layers_bfs(Gg, N, L)
    alphas = np.array([1.0], dtype=np.float64)

    stats_file = f"STATS_L1_{topo_name}Z{z_val}.csv"
    trans_file = f"TRANSITORY_L1_{topo_name}Z{z_val}.csv"

    for label, rule_type, k_val in CASOS:
        print(f"  > Caso: {label}")
        for b in B_RANGE:
            seeds = [12345 + 1000*r for r in range(REPS)]
            data = Parallel(n_jobs=-1)(delayed(run_simulation_exhaustive_v2)(
                af, ao, deg, inf_idx, inf_cnt, alphas, theta, b, steps, rule_type, k_val, seeds[r], sampling_rate
            ) for r in range(REPS))
            
            rhos = np.array([x[0] for x in data])
            times = np.array([x[1] for x in data])
            stops = np.array([x[2] for x in data])
            hists = np.array([x[3] for x in data])
            
            # Stats (avec codes d'arrêt)
            pd.DataFrame([{
                "Label": label, "b": b, 
                "rho_mean": rhos.mean(), "rho_std": rhos.std(),
                "Tstop_mean": times.mean(),
                "Stop_absorb_frac": np.mean(stops == 0),
                "Stop_std_frac": np.mean(stops == 1),
                "Stop_max_frac": np.mean(stops == 2)
            }]).to_csv(stats_file, mode='a', index=False, header=not os.path.exists(stats_file))
            
            # Transitoire
            mean_hist = hists.mean(axis=0)
            df_trans = pd.DataFrame({
                "Label": label, "b": b, 
                "step": np.arange(len(mean_hist))*sampling_rate, 
                "rho_t": mean_hist
            })
            df_trans.to_csv(trans_file, mode='a', index=False, header=not os.path.exists(trans_file))

print("\n✅ Simulation complète terminée.")